# Product Classification

## Evaluate your model


In [4]:
import os
os.chdir("/home/gianniif/ecommerce-predictor/")
from sklearn.metrics import accuracy_score, classification_report

from scripts import utils
from scripts import efficientnet

## Experiment settings

Set here the two variables in the following way:

- **CONFIG_YML:** assign the path to the config.yml file used for the experiment you want to evaluate
- **WEIGHTS:** assign the path to the model weights (.h5 file) you want to evaluate


In [9]:
# Modify this
CONFIG_YML = "/home/gianniif/ecommerce-predictor/experiments/exp4/exp42.yml"
WEIGHTS = "/home/gianniif/ecommerce-predictor/experiments/exp4/model.06-2.0593.h5"

In [6]:
# (!) Don't touch this cell, will load the config file specified before
config = utils.load_config(CONFIG_YML)
config

{'seed': 123,
 'data': {'directory': '/home/gianniif/ecommerce-predictor/data_splitted/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 32},
 'model': {'weights': 'imagenet',
  'input_shape': [224, 224, 3],
  'classes': 213,
  'dropout_rate': 0.2,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal'}}},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.001}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 150,
  'callbacks': {'model_checkpoint': {'filepath': '/home/gianniif/ecommerce-predictor/experiments/exp4/model.{epoch:02d}-{val_loss:.4f}.h5',
    'save_best_only': True},
   'tensor_board': {'log_dir': '/home/app/src/experiments/exp4/logs'}}}}

In [7]:
# (!) Don't touch this cell, will use the config file to infer the class names
#     and also to load the corresponding testing dataset.
#     If you get an error, you must review your data/code.

MODEL_CLASSES = utils.get_class_names(config)

if len(MODEL_CLASSES) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your data!"
    )

_dirname, _ = os.path.split(config['data']['directory'])
TEST_FOLDER = os.path.join(_dirname, 'test')

if not os.path.exists(TEST_FOLDER):
    raise ValueError("'test' folder not found!")
    
if len(os.listdir(TEST_FOLDER)) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your testing dataset!"
    )
    
if set(os.listdir(TEST_FOLDER)) != set(MODEL_CLASSES):
    raise ValueError(
        "The name of the subfolders inside your test set "
        "doesn't match with the model classes!"
    )

## Load your model

Use `efficietnet.create_model()` and remember to properly setup the model weights!

Assign the model to the variable `cnn_model`.


In [10]:
# Complete this
# TODO
cnn_model = efficientnet.create_model(weights=WEIGHTS)

# It should print your model correctly
print(cnn_model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
efficientnetv2-b0 (Functiona (None, 1280)              5919312   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 213)               272853    
Total params: 6,192,165
Trainable params: 6,131,557
Non-trainable params: 60,608
_________________________________________________________________
None


## Get predictions from testing dataset

In [11]:
# (!) Don't touch this cell, will use the loaded model and
#     the function utils.predict_from_folder() to get 
#     model predictions and the corresponding true labels
#     so we can measure the accuracy
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

predictions, labels = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)

if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/efficientnetv2-b0/stem_conv/Conv2D (defined at /home/gianniif/ecommerce-predictor/scripts/utils.py:156) ]] [Op:__inference_predict_function_12582]

Function call stack:
predict_function


In [8]:
# (!) Don't touch this cell

print(classification_report(y_true=labels, y_pred=predictions))


                                                        precision    recall  f1-score   support

                            AM General Hummer SUV 2000       0.84      0.95      0.89        44
                             Acura Integra Type R 2001       0.81      0.86      0.84        44
                                   Acura RL Sedan 2012       0.88      0.66      0.75        32
                                   Acura TL Sedan 2012       0.66      0.91      0.76        43
                                  Acura TL Type-S 2008       1.00      0.88      0.94        42
                                  Acura TSX Sedan 2012       0.77      0.75      0.76        40
                              Acura ZDX Hatchback 2012       0.92      0.62      0.74        39
              Aston Martin V8 Vantage Convertible 2012       0.53      0.73      0.62        45
                    Aston Martin V8 Vantage Coupe 2012       0.63      0.41      0.50        41
                  Aston Martin Virage C

In [9]:
# (!) Don't touch this cell

acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.7987!
